In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 新しいセクション

In [21]:
from skimage import io
import torch
from torch import nn
import torchvision.transforms as transforms
from PIL import Image

input="/content/drive/MyDrive/signate/タナチョー 部材の画像認識/train/000/0.jpg"
image = io.imread(input)

class ImageTransform():
  def __init__(self):
    self.data_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
  def __call__(self, img):
    return self.data_transform(img)

IT=ImageTransform()
tensor_img=IT(Image.fromarray(image))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tensor_img=tensor_img.unsqueeze(0).to(device)

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
model.classifier=nn.Linear(1024, 16,bias=True)
model.to(device)
load_path ="/content/drive/MyDrive/signate/タナチョー 部材の画像認識/test/category_color_model20220709_205149.pth"
if torch.cuda.is_available():
    model.load_state_dict(torch.load(load_path))
else:
    model.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))
model = model.eval()
category_color_model_output=model(tensor_img)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
model.classifier=nn.Linear(1024, 122,bias=True)
model.to(device)
load_path = "/content/drive/MyDrive/signate/タナチョー 部材の画像認識/test/model20220706_204326.pth"
if torch.cuda.is_available():
    model.load_state_dict(torch.load(load_path))
else:
    model.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))
model = model.eval()
model_output=model(tensor_img)
print(category_color_model_output,model_output)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


tensor([[ 1.2790, -0.9569,  0.6175, -0.6302, -0.6282, -1.2318, -0.3290,  2.8677,
          2.9647, -0.3625, -1.0875, -2.6111, -0.7639, -0.1415,  1.1913, -0.9498]],
       device='cuda:0', grad_fn=<AddmmBackward0>) tensor([[ 7.2688e+00,  1.9152e+00, -3.2463e-01,  1.0300e+00, -2.7080e-01,
          6.8408e-01, -2.0005e-01,  1.8899e+00, -1.0493e+00,  1.7427e-01,
         -3.0288e-01,  2.4924e+00, -3.9594e-01,  3.3056e-01, -1.9115e+00,
          6.9823e+00, -8.0050e-01, -2.9430e-01, -1.1936e+00, -2.5053e-01,
         -4.3599e-01, -7.9199e-01,  6.9291e-01, -4.5397e-01,  1.9666e-01,
         -1.7488e-01,  1.6169e+00, -2.1142e-01,  1.5380e+00, -1.1996e+00,
          6.0954e-01,  1.6594e+00,  2.5842e+00, -1.9933e-01,  9.6935e-01,
          1.6403e-01,  1.2224e+00, -1.7831e-01, -7.8017e-01, -6.5535e-01,
          7.8778e-01,  1.6146e+00, -8.1748e-02,  1.7029e+00,  7.0823e-02,
          1.5530e-01, -5.0977e-03,  2.9723e-01,  4.0680e-01,  2.9617e+00,
         -1.0981e+00, -4.6410e-01, -3.8109e-01

In [23]:
category_color_model_output

tensor([[ 1.2790, -0.9569,  0.6175, -0.6302, -0.6282, -1.2318, -0.3290,  2.8677,
          2.9647, -0.3625, -1.0875, -2.6111, -0.7639, -0.1415,  1.1913, -0.9498]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

# データ処理

In [24]:
import pandas as pd
import json

df=pd.read_json('/content/drive/MyDrive/signate/タナチョー 部材の画像認識/train_meta.json').T
df["category_color"]=df.loc[:,"category"]+df.loc[:,"color"]

def encode_df_label(df,col:str):
  from sklearn.preprocessing import LabelEncoder
  le=LabelEncoder()
  df['encoded_'+col]=le.fit_transform(df[col].values)
  return df

df=encode_df_label(df,"category")
df=encode_df_label(df,"color")
df=encode_df_label(df,"category_color")
df

,category,color,category_color,encoded_category,encoded_color,encoded_category_color
0,クレセント,YW,クレセントYW,0,10,8
1,クレセント,A3,クレセントA3,0,0,0
2,クレセント,DG,クレセントDG,0,3,3
3,クレセント,YS,クレセントYS,0,9,7
4,戸車,YS,戸車YS,1,9,14
...,...,...,...,...,...,...
117,戸車,YS,戸車YS,1,9,14
118,戸車,YS,戸車YS,1,9,14
119,戸車,YS,戸車YS,1,9,14
120,クレセント,WM,クレセントWM,0,6,4


In [34]:
df['encoded_category_color'].to_csv('/content/drive/MyDrive/signate/タナチョー 部材の画像認識/category_color.csv')

In [37]:
category_color=df['encoded_category_color'].to_list()

In [44]:
category_color_model_output,model_output=category_color_model_output.tolist(),model_output.tolist()
tmp=[0]*122
for i in range(len(category_color)):
    tmp[i]=category_color_model_output[0][category_color[i]]
category_color_model_output=tmp


In [55]:
import numpy as np
added=np.array(category_color_model_output)+np.array(model_output)

In [56]:
np.argsort(added[0])[::-1]

array([  0,  15,  85,  49,  11,  89,  76,  80,   7,  55,  31,  41,  78,
        99,  65,   3,  32,  96,  22,  56,  30,  59,  79,  97,   1,   9,
        35,  44,  26,  92,  33,  93,  40,  84,   5,  57,  69,  94,  82,
        48,  34,  62,  13,  91,  24,  63,  70,  45,  42,  46,  28,  43,
        67,  19,  60,  25,  37,   6,  68,  27,  77,   4,  47,  17,  10,
        58, 104,  23,  98,  12,  20,  51,  38,  16,  36,  95,  87,  53,
        61,  66, 119,  52, 111, 117, 108,  83, 121,   2,  39, 113,  75,
        88,  72,  81,  74,  71, 118, 116, 105, 101, 102,  64,  21, 100,
         8,  90,  18,  50, 115, 112,  29,  54,  86, 109, 114,  73,  14,
       120, 110, 107, 106, 103])

In [ ]:
df.loc[:,"encoded_category_color"].max()+1

16

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

#https://pytorch.org/vision/0.12/_modules/torchvision/datasets/folder.html
#https://pystyle.info/pytorch-how-to-create-custom-dataset-class/
class MyImageFolder(datasets.ImageFolder):
    # 要素を参照すると呼ばれる関数 
    def __init__(self,root,transform,label_df):
        super().__init__(root,transform)
        self.label_df=label_df
    def __getitem__(self, idx):
      sample, target=super().__getitem__(idx)
      return sample, self.label_df.iloc[target]


#https://pystyle.info/pytorch-list-of-transforms/
class ImageTransform():
  def __init__(self):
    self.data_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomRotation(degrees=180),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.9),
        #transforms.GaussianBlur(kernel_size=3),
        transforms.ToTensor(),
        #transforms.RandomErasing(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
  def __call__(self, img):
    return self.data_transform(img)

In [ ]:
def train_step(train_X, train_y):
    train_X, train_y=train_X.to(device),train_y.to(device)

    # 訓練モードに設定
    model.train()

    # フォワードプロパゲーションで出力結果を取得
    #train_X                # 入力データ
    pred_y = model(train_X) # 出力結果
    #train_y                # 正解ラベル

    # 出力結果と正解ラベルから損失を計算し、勾配を求める
    optimizer.zero_grad()   # 勾配を0で初期化（※累積してしまうため要注意）
    loss = criterion(pred_y, train_y)     # 誤差（出力結果と正解ラベルの差）から損失を取得
    loss.backward()   # 逆伝播の処理として勾配を計算（自動微分）

    # 勾配を使ってパラメーター（重みとバイアス）を更新
    optimizer.step()  # 指定されたデータ分の最適化を実施 

    # 正解率を算出
    with torch.no_grad(): # 勾配は計算しないモードにする
        acc=0
        for i in range(len(train_y)):
          acc+=1 if torch.argmax(pred_y[i])==train_y[i] else 0

    # 損失と正解数をタプルで返す
    return (loss.item(), acc)  # ※item()=Pythonの数値

def valid_step(valid_X, valid_y):
    valid_X, valid_y=valid_X.to(device),valid_y.to(device)

    # 評価モードに設定（※dropoutなどの挙動が評価用になる）
    model.eval()
    
    # フォワードプロパゲーションで出力結果を取得
    #valid_X                # 入力データ
    pred_y = model(valid_X) # 出力結果
    #valid_y                # 正解ラベル

    # 出力結果と正解ラベルから損失を計算
    loss = criterion(pred_y, valid_y)     # 誤差（出力結果と正解ラベルの差）から損失を取得

    # 正解率を算出
    with torch.no_grad(): # 勾配は計算しないモードにする
        acc=0
        for i in range(len(valid_y)):
          acc+=1 if torch.argmax(pred_y[i])==valid_y[i] else 0

    # 損失と正解数をタプルで返す
    return (loss.item(), acc)  # ※item()=Pythonの数値

# 部材種類色分類モデル

In [ ]:
import torch
from torch import nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
model.classifier=nn.Linear(1024, df.loc[:,"encoded_category_color"].max()+1,bias=True)
model.to(device)

In [ ]:
#https://tzmi.hatenablog.com/entry/2020/03/05/222813
load_path = '/content/drive/MyDrive/signate/タナチョー 部材の画像認識/model/category_color/model20220708_134723.pth'
if torch.cuda.is_available():
  model.load_state_dict(torch.load(load_path))
else:
  model.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))

In [ ]:
import torch.optim as optim   # 「最適化」モジュールの別名定義

# 定数（学習方法設計時に必要となるもの）
LEARNING_RATE = 1e-3   # 学習率： 0.03
REGULARIZATION = 0.03  # 正則化率： 0.03

# オプティマイザーを作成（パラメーターと学習率も指定）
optimizer = optim.SGD(           # 最適化アルゴリズムに「SGD」を選択
    model.parameters(),          # 最適化で更新対象のパラメーター（重みやバイアス）
    lr=LEARNING_RATE,            # 更新時の学習率
    weight_decay=REGULARIZATION) # L2正則化（※不要な場合は0か省略）

# 変数（学習方法設計時に必要となるもの）
criterion = nn.CrossEntropyLoss()  # 損失関数



images=MyImageFolder("/content/drive/MyDrive/signate/タナチョー 部材の画像認識/train",ImageTransform(),df.loc[:,"encoded_category_color"])

batch_size = 64
images_size=len(images)
train_size = int((images_size * 0.8)//1)
dataset_train = Subset(images,list(range(0,train_size)))
dataset_valid   = Subset(images, list(range(train_size,images_size)))
print("train_len:",len(dataset_train)," vaild_len:",len(dataset_valid))

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size)
print("train_iter:",len(loader_train)," vaild_iter:",len(loader_valid))


from datetime import datetime
from pytz import timezone
save_path='/content/drive/MyDrive/signate/タナチョー 部材の画像認識/model/category_color/model'+datetime.now(timezone('Asia/Tokyo')).strftime('%Y%m%d_%H%M%S')+'.pth'



# 定数（学習／評価時に必要となるもの）
EPOCHS = 100         # エポック数： 100

# 変数（学習／評価時に必要となるもの）
avg_loss = 0.0           # 「訓練」用の平均「損失値」
avg_acc = 0.0            # 「訓練」用の平均「正解率」
avg_val_loss = 0.0       # 「評価」用の平均「損失値」
avg_val_acc = 0.0        # 「評価」用の平均「正解率」

# 損失の履歴を保存するための変数
train_history = []
valid_history = []

print("Start Training At "+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S'))

for epoch in range(EPOCHS):
    # forループ内で使う変数と、エポックごとの値リセット
    total_loss = 0.0     # 「訓練」時における累計「損失値」
    total_acc = 0.0      # 「訓練」時における累計「正解数」
    total_val_loss = 0.0 # 「評価」時における累計「損失値」
    total_val_acc = 0.0  # 「評価」時における累計「正解数」
    total_train = 0      # 「訓練」時における累計「データ数」
    total_valid = 0      # 「評価」時における累計「データ数」

    for train_X, train_y in loader_train:
        # 【重要】1ミニバッチ分の「訓練」を実行
        loss,acc = train_step(train_X, train_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_loss += loss          # 訓練用の累計損失値
        total_acc += acc            # 訓練用の累計正解数
        total_train += len(train_y) # 訓練データの累計数
            
    for valid_X, valid_y in loader_valid:
        # 【重要】1ミニバッチ分の「評価（精度検証）」を実行
        val_loss,val_acc = valid_step(valid_X, valid_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_val_loss += val_loss  # 評価用の累計損失値
        total_val_acc += val_acc    # 評価用の累計正解数
        total_valid += len(valid_y) # 訓練データの累計数

    # ミニバッチ単位で累計してきた損失値や正解率の平均を取る
    n = epoch + 1                             # 処理済みのエポック数
    avg_loss = total_loss / n                 # 訓練用の平均損失値
    avg_acc = total_acc / total_train         # 訓練用の平均正解率
    avg_val_loss = total_val_loss / n         # 訓練用の平均損失値
    avg_val_acc = total_val_acc / total_valid # 訓練用の平均正解率

    # グラフ描画のために損失の履歴を保存する
    train_history.append(avg_loss)
    valid_history.append(avg_val_loss)

    # 損失や正解率などの情報を表示
    print(f'[Epoch {epoch+1:3d}/{EPOCHS:3d}]' \
          f' loss: {avg_loss:.5f}, acc: {avg_acc:.5f}' \
          f' val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f} At '+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S'))
    
    # save model
    torch.save(model.state_dict(), save_path)
print('Finished Training')

import matplotlib.pyplot as plt
# 学習結果（損失）のグラフを描画
epochs = len(train_history)
plt.plot(range(epochs), train_history, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_history, marker='.', label='loss (Validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

train_len: 910  vaild_len: 228
train_iter: 15  vaild_iter: 4
Start Training At 07/09 20:51:49
[Epoch   1/100] loss: 30.55608, acc: 0.38681 val_loss: 8.97498, val_acc: 0.35965 At 07/09 20:56:00
[Epoch   2/100] loss: 14.27632, acc: 0.42088 val_loss: 4.43012, val_acc: 0.35088 At 07/09 21:00:09
[Epoch   3/100] loss: 9.19810, acc: 0.45275 val_loss: 2.92636, val_acc: 0.38158 At 07/09 21:04:17
[Epoch   4/100] loss: 6.60929, acc: 0.47143 val_loss: 2.18681, val_acc: 0.37281 At 07/09 21:08:26
[Epoch   5/100] loss: 5.26508, acc: 0.50769 val_loss: 1.70167, val_acc: 0.39035 At 07/09 21:12:32
[Epoch   6/100] loss: 4.26939, acc: 0.53516 val_loss: 1.41586, val_acc: 0.39035 At 07/09 21:16:38
[Epoch   7/100] loss: 3.55575, acc: 0.52967 val_loss: 1.20829, val_acc: 0.40789 At 07/09 21:20:46
[Epoch   8/100] loss: 3.06512, acc: 0.54835 val_loss: 1.03584, val_acc: 0.39912 At 07/09 21:24:54
[Epoch   9/100] loss: 2.60943, acc: 0.55165 val_loss: 0.93343, val_acc: 0.39474 At 07/09 21:29:01
[Epoch  10/100] loss: 

KeyboardInterrupt: ignored

# 部材種類分類モデル
DenseNet
https://pytorch.org/hub/pytorch_vision_densenet/
https://atmarkit.itmedia.co.jp/ait/articles/2002/20/news029.html

In [ ]:
import torch
from torch import nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
model.classifier=nn.Linear(1024,df.loc[:,"encoded_category"].max()+1,bias=True)
model.to(device)

In [ ]:
#https://tzmi.hatenablog.com/entry/2020/03/05/222813
load_path = '/content/drive/MyDrive/signate/タナチョー 部材の画像認識/model/category/model20220708_134723.pth'
if torch.cuda.is_available():
  model.load_state_dict(torch.load(load_path))
else:
  model.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))

In [ ]:
import torch.optim as optim   # 「最適化」モジュールの別名定義

# 定数（学習方法設計時に必要となるもの）
LEARNING_RATE = 1e-3   # 学習率： 0.03
REGULARIZATION = 0.03  # 正則化率： 0.03

# オプティマイザーを作成（パラメーターと学習率も指定）
optimizer = optim.SGD(           # 最適化アルゴリズムに「SGD」を選択
    model.parameters(),          # 最適化で更新対象のパラメーター（重みやバイアス）
    lr=LEARNING_RATE,            # 更新時の学習率
    weight_decay=REGULARIZATION) # L2正則化（※不要な場合は0か省略）

# 変数（学習方法設計時に必要となるもの）
criterion = nn.CrossEntropyLoss()  # 損失関数

images=MyImageFolder("/content/drive/MyDrive/signate/タナチョー 部材の画像認識/train",ImageTransform(),df.loc[:,"encoded_category"])

batch_size = 64
images_size=len(images)
train_size = int((images_size * 0.8)//1)
dataset_train = Subset(images,list(range(0,train_size)))
dataset_valid   = Subset(images, list(range(train_size,images_size)))
print("train_len:",len(dataset_train)," vaild_len:",len(dataset_valid))

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size)
print("train_iter:",len(loader_train)," vaild_iter:",len(loader_valid))

train_len: 910  vaild_len: 228
train_iter: 15  vaild_iter: 4


In [ ]:
from datetime import datetime
from pytz import timezone
save_path='/content/drive/MyDrive/signate/タナチョー 部材の画像認識/model/category/model'+datetime.now(timezone('Asia/Tokyo')).strftime('%Y%m%d_%H%M%S')+'.pth'



# 定数（学習／評価時に必要となるもの）
EPOCHS = 50        # エポック数： 100

# 変数（学習／評価時に必要となるもの）
avg_loss = 0.0           # 「訓練」用の平均「損失値」
avg_acc = 0.0            # 「訓練」用の平均「正解率」
avg_val_loss = 0.0       # 「評価」用の平均「損失値」
avg_val_acc = 0.0        # 「評価」用の平均「正解率」

# 損失の履歴を保存するための変数
train_history = []
valid_history = []

print("Start Training At "+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S'))

for epoch in range(EPOCHS):
    # forループ内で使う変数と、エポックごとの値リセット
    total_loss = 0.0     # 「訓練」時における累計「損失値」
    total_acc = 0.0      # 「訓練」時における累計「正解数」
    total_val_loss = 0.0 # 「評価」時における累計「損失値」
    total_val_acc = 0.0  # 「評価」時における累計「正解数」
    total_train = 0      # 「訓練」時における累計「データ数」
    total_valid = 0      # 「評価」時における累計「データ数」

    for train_X, train_y in loader_train:
        # 【重要】1ミニバッチ分の「訓練」を実行
        loss,acc = train_step(train_X, train_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_loss += loss          # 訓練用の累計損失値
        total_acc += acc            # 訓練用の累計正解数
        total_train += len(train_y) # 訓練データの累計数
            
    for valid_X, valid_y in loader_valid:
        # 【重要】1ミニバッチ分の「評価（精度検証）」を実行
        val_loss,val_acc = valid_step(valid_X, valid_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_val_loss += val_loss  # 評価用の累計損失値
        total_val_acc += val_acc    # 評価用の累計正解数
        total_valid += len(valid_y) # 訓練データの累計数

    # ミニバッチ単位で累計してきた損失値や正解率の平均を取る
    n = epoch + 1                             # 処理済みのエポック数
    avg_loss = total_loss / n                 # 訓練用の平均損失値
    avg_acc = total_acc / total_train         # 訓練用の平均正解率
    avg_val_loss = total_val_loss / n         # 訓練用の平均損失値
    avg_val_acc = total_val_acc / total_valid # 訓練用の平均正解率

    # グラフ描画のために損失の履歴を保存する
    train_history.append(avg_loss)
    valid_history.append(avg_val_loss)

    # 損失や正解率などの情報を表示
    print(f'[Epoch {epoch+1:3d}/{EPOCHS:3d}]' \
          f' loss: {avg_loss:.5f}, acc: {avg_acc:.5f}' \
          f' val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f} At '+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S'))
    
    # save model
    torch.save(model.state_dict(), save_path)
print('Finished Training')

import matplotlib.pyplot as plt
# 学習結果（損失）のグラフを描画
epochs = len(train_history)
plt.plot(range(epochs), train_history, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_history, marker='.', label='loss (Validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

Start Training At 07/08 17:18:56
[Epoch   1/ 50] loss: 1.38518, acc: 0.97473 val_loss: 0.85260, val_acc: 0.91228 At 07/08 17:37:48
[Epoch   2/ 50] loss: 0.68304, acc: 0.98242 val_loss: 0.37624, val_acc: 0.92982 At 07/08 17:52:26
[Epoch   3/ 50] loss: 0.49089, acc: 0.97253 val_loss: 0.24584, val_acc: 0.91228 At 07/08 18:07:07
[Epoch   4/ 50] loss: 0.34204, acc: 0.97363 val_loss: 0.19102, val_acc: 0.92105 At 07/08 18:21:46
[Epoch   5/ 50] loss: 0.24355, acc: 0.98132 val_loss: 0.15876, val_acc: 0.91228 At 07/08 18:36:31
[Epoch   6/ 50] loss: 0.18875, acc: 0.98132 val_loss: 0.10381, val_acc: 0.92982 At 07/08 18:51:15
[Epoch   7/ 50] loss: 0.19748, acc: 0.97253 val_loss: 0.10009, val_acc: 0.92544 At 07/08 19:06:00
[Epoch   8/ 50] loss: 0.14051, acc: 0.98132 val_loss: 0.09096, val_acc: 0.90789 At 07/08 19:20:50
[Epoch   9/ 50] loss: 0.14388, acc: 0.97802 val_loss: 0.07896, val_acc: 0.91228 At 07/08 19:35:38
[Epoch  10/ 50] loss: 0.11568, acc: 0.98132 val_loss: 0.07937, val_acc: 0.92105 At 07

KeyboardInterrupt: ignored

# 部材色分類モデル

In [ ]:
import torch
from torch import nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
model.classifier=nn.Linear(1024, df.loc[:,"encoded_color"].max()+1,bias=True)
model.to(device)

In [ ]:
#https://tzmi.hatenablog.com/entry/2020/03/05/222813
load_path = '/content/drive/MyDrive/signate/タナチョー 部材の画像認識/model/color/model20220708_134723.pth'
if torch.cuda.is_available():
  model.load_state_dict(torch.load(load_path))
else:
  model.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))

In [ ]:
import torch.optim as optim   # 「最適化」モジュールの別名定義

# 定数（学習方法設計時に必要となるもの）
LEARNING_RATE = 1e-3   # 学習率： 0.03
REGULARIZATION = 0.03  # 正則化率： 0.03

# オプティマイザーを作成（パラメーターと学習率も指定）
optimizer = optim.SGD(           # 最適化アルゴリズムに「SGD」を選択
    model.parameters(),          # 最適化で更新対象のパラメーター（重みやバイアス）
    lr=LEARNING_RATE,            # 更新時の学習率
    weight_decay=REGULARIZATION) # L2正則化（※不要な場合は0か省略）

# 変数（学習方法設計時に必要となるもの）
criterion = nn.CrossEntropyLoss()  # 損失関数



images=MyImageFolder("/content/drive/MyDrive/signate/タナチョー 部材の画像認識/train",ImageTransform(),df.loc[:,"encoded_color"])

batch_size = 64
images_size=len(images)
train_size = int((images_size * 0.8)//1)
dataset_train = Subset(images,list(range(0,train_size)))
dataset_valid   = Subset(images, list(range(train_size,images_size)))
print("train_len:",len(dataset_train)," vaild_len:",len(dataset_valid))

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size)
print("train_iter:",len(loader_train)," vaild_iter:",len(loader_valid))


from datetime import datetime
from pytz import timezone
save_path='/content/drive/MyDrive/signate/タナチョー 部材の画像認識/model/color/model'+datetime.now(timezone('Asia/Tokyo')).strftime('%Y%m%d_%H%M%S')+'.pth'



# 定数（学習／評価時に必要となるもの）
EPOCHS = 200         # エポック数： 100

# 変数（学習／評価時に必要となるもの）
avg_loss = 0.0           # 「訓練」用の平均「損失値」
avg_acc = 0.0            # 「訓練」用の平均「正解率」
avg_val_loss = 0.0       # 「評価」用の平均「損失値」
avg_val_acc = 0.0        # 「評価」用の平均「正解率」

# 損失の履歴を保存するための変数
train_history = []
valid_history = []

print("Start Training At "+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S'))

for epoch in range(EPOCHS):
    # forループ内で使う変数と、エポックごとの値リセット
    total_loss = 0.0     # 「訓練」時における累計「損失値」
    total_acc = 0.0      # 「訓練」時における累計「正解数」
    total_val_loss = 0.0 # 「評価」時における累計「損失値」
    total_val_acc = 0.0  # 「評価」時における累計「正解数」
    total_train = 0      # 「訓練」時における累計「データ数」
    total_valid = 0      # 「評価」時における累計「データ数」

    for train_X, train_y in loader_train:
        # 【重要】1ミニバッチ分の「訓練」を実行
        loss,acc = train_step(train_X, train_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_loss += loss          # 訓練用の累計損失値
        total_acc += acc            # 訓練用の累計正解数
        total_train += len(train_y) # 訓練データの累計数
            
    for valid_X, valid_y in loader_valid:
        # 【重要】1ミニバッチ分の「評価（精度検証）」を実行
        val_loss,val_acc = valid_step(valid_X, valid_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_val_loss += val_loss  # 評価用の累計損失値
        total_val_acc += val_acc    # 評価用の累計正解数
        total_valid += len(valid_y) # 訓練データの累計数

    # ミニバッチ単位で累計してきた損失値や正解率の平均を取る
    n = epoch + 1                             # 処理済みのエポック数
    avg_loss = total_loss / n                 # 訓練用の平均損失値
    avg_acc = total_acc / total_train         # 訓練用の平均正解率
    avg_val_loss = total_val_loss / n         # 訓練用の平均損失値
    avg_val_acc = total_val_acc / total_valid # 訓練用の平均正解率

    # グラフ描画のために損失の履歴を保存する
    train_history.append(avg_loss)
    valid_history.append(avg_val_loss)

    # 損失や正解率などの情報を表示
    print(f'[Epoch {epoch+1:3d}/{EPOCHS:3d}]' \
          f' loss: {avg_loss:.5f}, acc: {avg_acc:.5f}' \
          f' val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f} At '+datetime.now(timezone('Asia/Tokyo')).strftime('%m/%d %H:%M:%S'))
    
    # save model
    torch.save(model.state_dict(), save_path)
print('Finished Training')

import matplotlib.pyplot as plt
# 学習結果（損失）のグラフを描画
epochs = len(train_history)
plt.plot(range(epochs), train_history, marker='.', label='loss (Training data)')
plt.plot(range(epochs), valid_history, marker='.', label='loss (Validation data)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

train_len: 910  vaild_len: 228
train_iter: 15  vaild_iter: 4
Start Training At 07/08 21:41:13
[Epoch   1/200] loss: 27.12253, acc: 0.48132 val_loss: 7.40855, val_acc: 0.43860 At 07/08 21:56:47
[Epoch   2/200] loss: 10.60821, acc: 0.61758 val_loss: 3.72420, val_acc: 0.43860 At 07/08 22:00:50
[Epoch   3/200] loss: 6.79712, acc: 0.61758 val_loss: 2.43883, val_acc: 0.44298 At 07/08 22:04:50
[Epoch   4/200] loss: 4.91297, acc: 0.61758 val_loss: 1.82631, val_acc: 0.44298 At 07/08 22:08:51
[Epoch   5/200] loss: 3.86994, acc: 0.61868 val_loss: 1.44759, val_acc: 0.43860 At 07/08 22:12:50
[Epoch   6/200] loss: 3.06532, acc: 0.61868 val_loss: 1.18883, val_acc: 0.44298 At 07/08 22:16:51
[Epoch   7/200] loss: 2.64666, acc: 0.62198 val_loss: 1.02826, val_acc: 0.43860 At 07/08 22:20:51
[Epoch   8/200] loss: 2.28720, acc: 0.62418 val_loss: 0.90240, val_acc: 0.44298 At 07/08 22:24:52
[Epoch   9/200] loss: 1.98481, acc: 0.62527 val_loss: 0.77068, val_acc: 0.45614 At 07/08 22:28:51
[Epoch  10/200] loss: 

In [ ]:

for valid_X, valid_y in loader_valid:
        # 【重要】1ミニバッチ分の「評価（精度検証）」を実行
        val_loss,val_acc = valid_step(valid_X, valid_y)

        # 取得した損失値と正解率を累計値側に足していく
        total_val_loss += val_loss  # 評価用の累計損失値
        total_val_acc += val_acc    # 評価用の累計正解数
        total_valid += len(valid_y) # 訓練データの累計数

def valid_step(valid_X, valid_y):
    valid_X, valid_y=valid_X.to(device),valid_y.to(device)

    # 評価モードに設定（※dropoutなどの挙動が評価用になる）
    model.eval()
    
    # フォワードプロパゲーションで出力結果を取得
    #valid_X                # 入力データ
    pred_y = model(valid_X) # 出力結果
    #valid_y                # 正解ラベル

    # 出力結果と正解ラベルから損失を計算
    loss = criterion(pred_y, valid_y)     # 誤差（出力結果と正解ラベルの差）から損失を取得

    # 正解率を算出
    with torch.no_grad(): # 勾配は計算しないモードにする
        acc=0
        for i in range(len(valid_y)):
          acc+=1 if torch.argmax(pred_y[i])==valid_y[i] else 0

    # 損失と正解数をタプルで返す
    return (loss.item(), acc)  # ※item()=Pythonの数値

# 部材型番分類モデル

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
class TestTransform():
  def __init__(self):
    self.data_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
    ])
  def __call__(self, img):
    return self.data_transform(img)
test_images=datasets.ImageFolder( "/content/drive/MyDrive/signate/タナチョー 部材の画像認識/train", transform = TestTransform())
test_images[0]

(<PIL.Image.Image image mode=RGB size=224x224 at 0x7F67683CAC10>, 0)

In [ ]:

!kill -9 $(lsof -t)